In [ ]:
%pip install openai chromadb==0.3.29 langchain tiktoken
%pip install -U langchain-community
%pip install faiss-cpu
%pip install sentence-transformers

# Import necessary libraries after resolving conflicts and installing dependencies
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from transformers import pipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import SimpleSequentialChain
# Import necessary libraries
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.docstore import InMemoryDocstore
from transformers import pipeline
import numpy as np
import faiss


import os

# List input data files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**Simple chatbot using Google model from Huggingface**

In [ ]:
from transformers import pipeline
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

# Choose and load a Hugging Face model for text generation
model_name = "google/flan-t5-large"  # Model suitable for text-to-text generation tasks
hf_pipeline = pipeline("text2text-generation", model=model_name, clean_up_tokenization_spaces=True)

# Wrap the Hugging Face pipeline using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a PromptTemplate
prompt_template = PromptTemplate(input_variables=["question"], template="{question}")

# Create a LangChain LLMChain with the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain with an input
response = chain.run({"question": "What is the capital of the USA?"})
print(response)


In [ ]:

# Create Sample Text Documents as LangChain Documents
sample_texts = [
    "Washington, D.C., formally the District of Columbia, is the capital city of the United States. It is located on the east bank of the Potomac River and was founded after the American Revolution to serve as the nation's capital.",
    "The United States of America (USA) is a country primarily located in North America. It consists of 50 states, a federal district, and several territories.",
    "San Francisco is a major city in California, known for its Golden Gate Bridge, cable cars, and colorful Victorian houses."
]

# Convert sample texts into LangChain Document objects
documents = [Document(page_content=text) for text in sample_texts]

# Chunk Documents
# Split documents into chunks for better retrieval performance
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunked_documents = text_splitter.split_documents(documents)

# Index the Documents
# Use Hugging Face embeddings for indexing
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create an index using FAISS for efficient similarity search
index = FAISS.from_documents(chunked_documents, embedding_model)

# Set Up the RAG Pipeline
# Choose and load a Hugging Face model for text generation
model_name = "google/flan-t5-large"  # Model suitable for text-to-text generation tasks
hf_pipeline = pipeline("text2text-generation", model=model_name)

# Wrap the Hugging Face pipeline using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a LangChain RetrievalQA chain that combines document retrieval with text generation
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 'stuff' or 'map_reduce' 
    retriever=index.as_retriever()
)

#Run the RAG Chain with an Input Question
response = rag_chain.run({"query": "What is the capital of the USA?"})
print(response)

**Next we will be using Chroma DB to store vectorised documents and also we will use SimpleSequentialChain**

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create Sample Text Documents as LangChain Documents
sample_texts = [
    "Washington, D.C., formally the District of Columbia, is the capital city of the United States. It is located on the east bank of the Potomac River and was founded after the American Revolution to serve as the nation's capital.",
    "The United States of America (USA) is a country primarily located in North America. It consists of 50 states, a federal district, and several territories.",
    "San Francisco is a major city in California, known for its Golden Gate Bridge, cable cars, and colorful Victorian houses.",
    "Harry Potter is a series of seven fantasy novels written by British author J.K. Rowling. The series chronicles the lives of a young wizard, Harry Potter, and his friends as they attend Hogwarts School of Witchcraft and Wizardry."
]

# Convert sample texts into LangChain Document objects
documents = [Document(page_content=text) for text in sample_texts]

# Chunk Documents
# Split documents into chunks for better retrieval performance
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunked_documents = text_splitter.split_documents(documents)

# Index the Documents using ChromaDB
# Use Hugging Face embeddings for indexing
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize ChromaDB vector store
chroma_db = Chroma.from_documents(documents=chunked_documents, embedding=embedding_model)

# Create a retriever from ChromaDB
retriever = chroma_db.as_retriever()

#  Set Up the RAG Pipeline
# Choose and load a Hugging Face model for text generation
model_name = "google/flan-t5-large"  # Model suitable for text-to-text generation tasks
hf_pipeline = pipeline("text2text-generation", model=model_name, clean_up_tokenization_spaces=True)  # Set to avoid FutureWarning

# Wrap the Hugging Face pipeline using LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a PromptTemplate for generating responses
prompt_template = PromptTemplate(input_variables=["context", "question"], template="{context}\n\n{question}")

# Chain Setup: Chain should properly retrieve and pass context
def retrieve_and_format(question):
    retrieved_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    return {"context": context, "question": question}

# Setup the pipeline correctly using components
chain = (
    RunnablePassthrough()  # This will pass the input question to the next step
    | retrieve_and_format  # Retrieve and format documents to create context
    | prompt_template  # Use the prompt template to structure the input
    | llm  # Generate a response using the language model
    | StrOutputParser() # Parse the output as a string
)

# Run the Chain Pipeline with the New Question
question = "Who is the author of Harry Potter?"
response = chain.invoke(question)
print(response)

**Concept of Memory** 

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory1 = ConversationBufferMemory( memory_key="history") 

memory1.save_context({"input": "hey"}, {"output": "wassup"}) 

print(f"memory1 ==> {memory1.load_memory_variables({})}")

memory2 = ConversationBufferMemory( memory_key="chat_history") 

memory2.save_context({"input": "hi"}, {"output": "whats are you doing?"}) 

print(f"memory2 ==> {memory2.load_memory_variables({})}")

In [ ]:



# Load the Hugging Face LLM model
hf_pipeline = pipeline('text-generation', model='gpt2', max_length=256, max_new_tokens=50)

# Wrap the pipeline in a HuggingFacePipeline for compatibility with LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Set up the embedding model for vector storage
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Sample documents for testing
documents = [
    Document(page_content="Paris is the capital of France."),
    Document(page_content="France is a country in Europe."),
    Document(page_content="The French Revolution was a period of social and political upheaval in France.")
]

#  Embed the documents
embedded_docs = [(doc, embedding_model.embed_query(doc.page_content)) for doc in documents]
doc_texts = [doc.page_content for doc in documents]
embeddings = [embedding_model.embed_query(text) for text in doc_texts]

#  Initialize the FAISS index
dimension = len(embeddings[0])  # Assuming all embeddings have the same length
index = faiss.IndexFlatL2(dimension)  # Using L2 (Euclidean) distance
index.add(np.array(embeddings))

# Initialize the FAISS vector store with InMemoryDocstore
docstore = InMemoryDocstore(dict(enumerate(documents)))
vector_store = FAISS(embedding_function=embedding_model, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(documents))})

# Initialize memory buffer with explicit output key
memory = ConversationBufferMemory(output_key='result')  # Set output_key to 'result'




In [ ]:
#  Create a retrieval-based QA chain
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Attach memory buffer to the chain
qa_chain.memory = memory

# Define a function to use the QA model with memory buffer
def query_with_memory(query: str):
    # Run the retrieval-based QA chain and get the results
    result = qa_chain({"query": query})
    
    # Extract the desired output ('result') to save in memory
    output_for_memory = {"result": result['result']}
    
    # Save only the relevant output key to memory
    memory.save_context({"query": query}, output_for_memory)
    
    # Retrieve memory for debugging or review
    memory_content = memory.load_memory_variables({})
    
    print("Memory Buffer Content:", memory_content)
    return result


In [ ]:

# Test the implementation
query = "What is the capital of France?"
response = query_with_memory(query)
print("Response:", response['result'])  # Use 'result' to match the correct key

# Add more queries to see memory buffer in action
query = "Tell me more about its history."
response = query_with_memory(query)
print("Response:", response['result'])  # Use 'result' to match the correct key